In [ ]:
import logging
from os import environ as env
from dotenv import load_dotenv

import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
import openai


logger = telebot.logger
telebot.logger.setLevel(logging.DEBUG)


load_dotenv()
bot = telebot.TeleBot(env["BOT_API_KEY"])
openai.api_key = env["OPENAI_API_KEY"]
user_id = env["USER_KEY"]


@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id,"Hi ✌️ ")
    

@bot.message_handler(commands=['img'])
def create_image(message):
    img_text = message.text[5:]
    bot.send_message(message.chat.id, "Choose image size", reply_markup=gen_image_markup(img_text))


def gen_image_markup(image_text):
    markup = InlineKeyboardMarkup()
    markup.row_width = 3
    markup.add(
        InlineKeyboardButton("256x256", callback_data="image_size|256|"+image_text),
        InlineKeyboardButton("512x512", callback_data="image_size|512|"+image_text),
        InlineKeyboardButton("1024x1024", callback_data="image_size|1024|"+image_text)
    )
    return markup


@bot.callback_query_handler(func=lambda call: call.data.startswith('image_size'))
def callback_query(call):
    imgSize = call.data.split('|')[1]
    imgSize = imgSize+'x'+imgSize
    
    imgText = call.data.split('|')[2]
    response = ''
    try:
        response = openai.Image.create(
            prompt=imgText,
            n=1,
            size=imgSize
        )
    except:
        bot.send_message(call.message.chat.id, "Can't generate that :(")
    else:
        bot.send_photo(call.message.chat.id, photo=response['data'][0]['url'])
    bot.answer_callback_query(call.id, "Done!")


import re
def first_num(text):
    match = re.search(r'\d+', text)
    return match.group()

    
def send_msg(chat_id, message):
    bot.send_message(chat_id, message)

@bot.message_handler(commands=['ask'])
@bot.message_handler(func=lambda message: True)
def get_response(message):
    response = ""
    # if message.text.startswith('/send'):
    #     if message.chat.id == my_id:
    #         chat_id = first_num(message.text)
    #         if chat_id is not None:
    #             send_msg(chat_id, message.text.split(chat_id)[1])
    #         else:
    #             bot.send_message(message.chat.id, 'Can not send a message')

    if message.text.startswith('>>>'):
      response = openai.Completion.create(
        engine="code-davinci-002",
        prompt=f'```\n{message.text[3:]}\n```',
        temperature=0,
        max_tokens=4000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n", ">>>"],
      )
    else:
      if "code" in message.text.lower() or "python" in message.text.lower():
          response = openai.Completion.create(
          engine="text-davinci-003",
          prompt=f'"""\n{message.text}\n"""',
          temperature=0,
          max_tokens=4000,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=['"""'],
        )
      else:
        response = openai.Completion.create(
          engine="text-davinci-003",
          prompt=f'"""\n{message.text}\n"""',
          temperature=0,
          max_tokens=2000,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=['"""'],
        )
    print(message.from_user.username, 'ask:', message.text)
    bot.send_message(message.chat.id, f'{response["choices"][0]["text"]}', parse_mode="None")
    print('response:', f'{response["choices"][0]["text"]}')
    
    
bot.infinity_polling()